In [1]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.5/898.5 kB 22.5 MB/s eta 0:00:00


In [2]:
!pip install deep_sort_realtime


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 77.8 MB/s eta 0:00:00


In [4]:
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import numpy as np

# Load YOLO model
model = YOLO('yolov8n.pt')  # Load YOLOv8 pre-trained model

# Initialize DeepSORT tracker
tracker = DeepSort(max_age=30, n_init=2, nn_budget=100)

# Load the video file
video_path = '/content/drive/MyDrive/detection_tracking_counting/video_4.mp4'
cap = cv2.VideoCapture(video_path)

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object
output_path = '/content/drive/MyDrive/detection_tracking_counting/output_full_video_cars.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

while cap.isOpened():  # Process the entire video
    ret, frame = cap.read()
    if not ret:
        break

    # Perform detection using YOLO
    results = model(frame)
    detections = []

    for result in results:
        for box in result.boxes.data:  # Extract bounding box info
            x1, y1, x2, y2, conf, cls = map(float, box[:6])
            cls = int(cls)  # Class ID
            if cls == 2:  # Only process cars (Class ID 2)
                detections.append([[x1, y1, x2, y2], conf])  # Format for DeepSORT

    # Update tracker with detections
    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue

        x1, y1, x2, y2 = map(int, track.to_ltwh())
        track_id = track.track_id

        # Draw bounding boxes and track ID
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)  # Green box for cars
        label = f'Car ID {track_id}'
        cv2.putText(frame, label, (x1, y1 - 15), cv2.FONT_HERSHEY_SIMPLEX,
                    1.0, (0, 0, 255), 3)  # Red text, larger font size

    # Write the frame to output
    out.write(frame)

cap.release()
out.release()

print("Detection and tracking complete. Output saved to:", output_path)



0: 384x640 15 persons, 6 cars, 15 motorcycles, 1 bus, 1 truck, 10.5ms
Speed: 2.7ms preprocess, 10.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 16 persons, 6 cars, 16 motorcycles, 1 bus, 1 truck, 9.0ms
Speed: 2.9ms preprocess, 9.0ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 16 persons, 6 cars, 17 motorcycles, 1 bus, 9.5ms
Speed: 3.2ms preprocess, 9.5ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 persons, 7 cars, 19 motorcycles, 1 bus, 1 truck, 8.7ms
Speed: 3.0ms preprocess, 8.7ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 16 persons, 6 cars, 21 motorcycles, 1 bus, 1 truck, 7.6ms
Speed: 3.0ms preprocess, 7.6ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 16 persons, 6 cars, 18 motorcycles, 1 bus, 1 truck, 8.6ms
Speed: 3.0ms preprocess, 8.6ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640